In [1]:
%run "catalog_common.py" 
ID_header('Open-FF:  Chemical Index', incl_links=True)
set_page_param()


<IPython.core.display.Javascript object>

,"Open-FF: Chemical Index This file generated on September 29, 2023 from data repository: cloud_repo_2023_09_20.",Sponsored by FracTracker Alliance


In [2]:
import pandas as pd
import numpy as np
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show as iShow
import itables.options as opt

df_cas = ana_set.Catalog_set(repo = repo_name, outdir='../common/').get_set(verbose=False)
gb2 = df_cas.groupby('bgCAS')['IngredientName'].agg(lambda x: x.value_counts().index[0])
gb2 = gb2.reset_index()
gb2.columns = ['bgCAS','comm_name']
df_cas = pd.merge(df_cas,gb2,on='bgCAS',how='left')
# df_cas.columns

<IPython.core.display.Javascript object>

In [3]:
#df_cas.columns

The interactive table below lists all chemicals (from the Open-FF data set) published in FracFocus after curation. Some chemicals are listed that have no records in the filtered set, but are in the full, raw data. Chemical identity is established using both CASNumber and IngredientName, when possible.  See the [CAS/Ingredient Index](Open-FF_CASNumber_and_IngredientName.html) for more detail on these identities. Below the table is an explanation of this table's columns.

The table is sort-able by any column (click a second time for reverse sort) and use the **Search** bar to limit what is shown in the table. 

In [4]:
YES = 'Yes'; NO = ''
chem_df = df_cas.groupby('bgCAS',as_index=False)[['UploadKey']].count()
chem_df = chem_df.rename({'UploadKey':'numRecords'},axis=1)
gb1 = df_cas[df_cas.in_std_filtered].groupby('bgCAS',as_index=False)[['UploadKey']].count()
chem_df = pd.merge(chem_df,gb1,on='bgCAS',how='left')
chem_df.fillna(0,inplace=True)
chem_df.numRecords = chem_df.UploadKey.astype('int').astype('str')+ '<br>'+ chem_df.numRecords.astype('str')
t = df_cas[(df_cas.calcMass>0)&(df_cas.in_std_filtered)].groupby('bgCAS',as_index=False)[['UploadKey']].count()
t = t.rename({'UploadKey':'numWithMass'},axis=1)

chem_df = pd.merge(chem_df,t,on='bgCAS',how='left')
chem_df.numWithMass.fillna(0,inplace=True)

t = df_cas.groupby('bgCAS')[['bgIngredientName','comm_name','is_on_TEDX','is_on_prop65',#'is_on_CWA_priority',
                             'is_on_CWA','is_on_DWSHA','is_on_PFAS_list','is_on_volatile_list',
                             'is_on_UVCB','is_on_diesel','is_on_AQ_CWA','is_on_HH_CWA','is_on_IRIS',
                             'is_on_NPDWR','rq_lbs',
                             'DTXSID']].first()
#t.rq_lbs = t.rq_lbs.map(lambda x: round_sig(x,2))
t.rq_lbs.fillna('  ',inplace=True)
chem_df = pd.merge(chem_df,t,on='bgCAS',how='left')

t = df_cas[(df_cas.calcMass>0)&(df_cas.in_std_filtered)].groupby('bgCAS',as_index=False)['calcMass'].apply(np.percentile,90)
t.calcMass = t.calcMass.map(lambda x: round_sig(x,3))
chem_df = pd.merge(chem_df,t,on='bgCAS',how='left')
chem_df.calcMass.fillna(0,inplace=True)

chem_df['Filtered Data'] = chem_df.bgCAS.map(lambda x: getDataLink(x))
chem_df['History'] = chem_df.bgCAS.map(lambda x: getCatLink(x,x))
chem_df['ChemID'] = chem_df.bgCAS.map(lambda x: getPubChemLink(x)) # Now through PubChem instead of ChemID
chem_df['EPA_ref'] = chem_df.DTXSID.map(lambda x: getCompToxRef(x))

chem_df['molecule'] = chem_df.bgCAS.map(lambda x: getMoleculeImg(x))
chem_df.molecule = chem_df.molecule # ?! why is this here?
chem_df['fingerprint'] = chem_df.bgCAS.map(lambda x: getFingerprintImg(x))

opt.classes = ['display','cell-border']
chem_df.bgIngredientName.fillna('non CAS',inplace=True)
chem_df['names'] = chem_df.bgIngredientName +'<br>----------<br>' + chem_df.comm_name
chem_df['just_cas'] = chem_df.bgCAS
chem_df.bgCAS = '<center><h3>'+chem_df.History+'</h3>'+chem_df.names+'</center>'
chem_df['ref'] = chem_df['Filtered Data']+'<br>'+chem_df.ChemID+'<br>'+chem_df.EPA_ref

chem_df['extrnl'] = np.where(chem_df.is_on_CWA,'CWA; ','')
chem_df.extrnl = np.where(chem_df.is_on_AQ_CWA,chem_df.extrnl+'AQ_CWA; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_HH_CWA,chem_df.extrnl+'HH_CWA; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_NPDWR,chem_df.extrnl+'NPDWR; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_DWSHA,chem_df.extrnl+'DWSHA; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_TEDX,chem_df.extrnl+'TEDX; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_prop65,chem_df.extrnl+'prop65; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_PFAS_list,chem_df.extrnl+'EPA_PFAS; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_volatile_list,chem_df.extrnl+'EPA_volatile; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_UVCB,chem_df.extrnl+'UVCB; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_diesel,chem_df.extrnl+'diesel; ',chem_df.extrnl)
chem_df.extrnl = np.where(chem_df.is_on_IRIS,chem_df.extrnl+'IRIS; ',chem_df.extrnl)

t = df_cas.groupby('bgCAS',as_index=False)['date'].min().rename({'date':'earliest_date',
                                                                 'bgCAS':'just_cas'},axis=1)
chem_df = pd.merge(chem_df,t,on='just_cas',how='left')
chem_df = chem_df[['bgCAS','ref','molecule',#'names',
                   #'bgIngredientName','comm_name',
                   'numRecords','numWithMass','calcMass','rq_lbs','fingerprint','extrnl',
                   'earliest_date']]
                   #'is_on_TEDX','is_on_prop65',
                   #'is_on_CWA_SDWA','is_on_PFAS_list','is_on_volatile_list']]
                   #'History',
                   #'Filtered Data','ChemID']] #,'eh_Class_L1','eh_Class_L2']]
chem_df = chem_df.rename({'bgCAS':'Material','bgIngredientName':'Name', 'comm_name':'Common Name',
                          'numRecords':'total num records',
                          'numWithMass':'num records with mass','calcMass':'Mass of 90th percentile (lbs)',
                          'extrnl':'on external lists','fingerprint':'fingerprint (see explanation below)',
                          'rq_lbs':'Reportable quant (lbs)'},
                          #'is_on_TEDX':'on TEDX list','is_on_prop65':'on Prop 65 list',
                          #'is_on_CWA_SDWA':'on CWA SDWA lists',
                          #'is_on_PFAS_list':'is PFAS or precursor','is_on_volatile_list':'on EPA volatile list'},
                          #'eh_Class_L1':'eh Class lvl 1','eh_Class_L2':'eh Class lvl 2'},
                         axis=1)
iShow(chem_df.reset_index(drop=True),maxBytes=0,columnDefs=[{"width": "100px", "targets": 0}])
#iShow(chem_df.reset_index(drop=True),maxBytes=0)


**Important note about chemical names**: most chemical compounds have a number of names they are known by.  For example, the chemical with the CAS registration number 111-76-4 is listed as:

| name | source |
| --- |--- |
|ethanol, 2-butoxy-|SciFinder (primary name)|
|Ethylene glycol monobutyl ether| EPA registry name|
|Ethylene glycol monobutyl ether (2-butoxyethanol)| Elsner/Hoelzer's paper|
|2-butoxyethanol| most common name in FracFocus|
| 2-BE |shorthand name|

In the table above, two names are given for each material, but many others are often used. That might be a problem if you are trying to find something by a different name.  If you are not having luck in the table above, try exploring the [synonyms index](Open-FF_Synonyms.html) for the most appropriate CAS Number.

|Explanation of columns in the index|
| :---: |

| Column      | Description |
| :----: | :-------- |
|*Material*| at top is the CAS registration number of the chemical. **Click on this link** to display summary of this chemical for all of FracFocus (filtered data - up to last major update). Below are two names for the material: the upper is used by SciFinder as the primary name and the lower is the most common reported name in FracFocus| 
|*ref*|Click on the "csv" link to a csv-formatted file that has all the filtered records for this chemical.  Click on "ChemID" link for the ChemID website for this chemical.  Click on "EPA: CompTox" for the summary page in that resource|
|*Molecule*| is the chemical structure (credit: ChemID)|
|*total num records* |indicates the number of records across the FracFocus data. A given chemical may appear more than once in a disclosure.  The upper number is the number of records in the **filtered** data, and the lower value is the number of records in the **unfiltered** data.|
|*num records with mass*|indicates the number of records with mass of this chemical in the **filtered** data. Without mass, a record only records presence in the disclosure.|
|*mass of 90th percentile (lbs)*| the top 10% of records in the **filtered** data have mass equal to or greater than this value|
|*Reportable quant (lbs)*|indicates the quantity of the chemical (in pounds), above which uses must be reported if spilled, based on [40 CFR § 302.4 - Hazardous substances and reportable quantities](https://www.law.cornell.edu/cfr/text/40/302.4). This is applicable throughout most industries however fracking is in general exempt from this requirement. We show it here to illustrate what those exemptions have given to the fracking industry. There are currently a total of about 100 chemicals on that list.
|*on external list* |indicates if the chemical is on one of the following lists.  Type the name of the list into the Search box to limit to those chemicals.<br> - **CWA**: indicates that the chemical is on the [Clean Water Act list](https://comptox.epa.gov/dashboard/chemical_lists/CWA311HS) as compiled in EPA's CompTox<br>- **AQ_CWA**: Comptox list of chemicals particularly hazardous to aquatic life<br>- **HH_CWA**: Comptox list of chemicals particularly hazardous to human health<br>- **DWSHA**: indicated that the chemical is on the EPA's [Drinking Water Safety and Health Advisory](https://comptox.epa.gov/dashboard/chemical_lists/EPADWS) list<br>- **NPDWR**: indicates that the chemical is on the [National Primary Drinking Water Regulations](https://www.epa.gov/ground-water-and-drinking-water/national-primary-drinking-water-regulations) list, compiled into a machine readable version by Angelica Fuiza and Gary Allison.<br>- **TEDX**: indicates that the chemical is on [The Endocrine Disruption Exchange](https://endocrinedisruption.org) list<br>- **prop_65**: indicates that the chemical is on [California's Proposition 65](https://oehha.ca.gov/proposition-65/proposition-65-list) list<br>- **EPA_PFAS**: indicates that the chemical is on EPA's comprehensive list of PFAS related compounds<br>- **EPA_volatile**: indicates that the chemical is on the EPA's volatile chemical list<br>- **UVCB**: indicates that the chemical is on the [TSCA list of "Unknown, Variable composition, Complex reaction products and Biological"](https://www.epa.gov/sites/default/files/2015-05/documents/uvcb.pdf) materials, meaning "these chemical substances cannot be represented by unique structures and molecular formulas."<br>- **IRIS**: indicates that the chemical is included in the [IRIS program](https://comptox.epa.gov/dashboard/chemical-lists/IRIS) that "identifies and characterizes the health hazards of chemicals found in the environment."<br>- **diesel**: indicates that the chemical is [on the EPA list that](https://www.epa.gov/uic/diesel-fuels-hydraulic-fracturing-dfhf) "represent the most appropriate interpretation of the statutory term 'diesel fuels' to use for permitting diesel fuels hydraulic fracturing under the UIC Program nationwide.|
|*fingerprint*:|This is a "hazard" summary based on the [EPA's ChemInformatics Dashboard](https://www.epa.gov/chemical-research/cheminformatics). For each chemical that the EPA has published summaries, Open-FF generates a single image that indicates level of hazard for EPA's twenty categories.  See the table below or click on the specific chemical's report (its CAS number) for a different display of these results. For many chemicals reported by the EPA on that site, some of those hazard categories cannot be evaluated because the data are limited.  See EPA's disclaimer for these data below or at their site.
|*earliest date*|indicates the date of the earliest recorded use of this chemical.  By sorting on this column, you can find the chemicals that have been long used and those that have only recently been added.|



## Hazard Fingerprints
### Categories
|Row| Col 1 | Col 2| Col 3| Col 4 |Col 5|
|:---| :---: | :---: | :---: | :---: | :---:|
|**Top** |*Human Health*<br>Oral |*Human Health*<br>Inhalation |*Human Health*<br>Dermal |*Human Health*<br>Carcinogenicity|*Human Health*<br>Genotoxicity Mutagenicity|
|**Second**|*Human Health*<br>Endocrine Disruption |*Human Health*<br>Reproductive |*Human Health*<br>Developmental |*Human Health*<br>Neurotoxicity:<br> Repeat Exposure |*Human Health*<br>Neurotoxicity:<br> Single Exposure|
|**Third**|*Human Health*<br>Systemic Toxicity:<br> Repeat Exposure |*Human Health*<br>Systemic Toxicity:<br> Single Exposure  |*Human Health*<br>Skin Sensitization  |*Human Health*<br>Skin Irritation|*Human Health*<br>Eye Irritation 
|**Bottom**|*Ecotoxicity*<br>Acute Aquatic Toxicity |*Ecotoxicity*<br>Chronic Aquatic Toxicity |*Fate*<br>Persistence |*Fate*<br>Bioaccumulation |*Fate*<br>Exposure | |

### Legend
|Icon| Toxicity |
| :---: | :---: |
| <center><img src="images/ci_icons/red_skull.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>Very High</H3>  |
| <center><img src="images/ci_icons/orange_exclamation.png" onerror="this.onerror=null; this.remove();" width="40"></center> |  <H3>High</H3>  |
| <center><img src="images/ci_icons/yellow-minus.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>Moderate</H3>  |
| <center><img src="images/ci_icons/green-minus.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>Low</H3>  |
| <center><img src="images/ci_icons/grey_question.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>Inconclusive</H3>  |
| <center><img src="images/ci_icons/grey_square.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>No Data</H3>  |

### EPA's disclaimer for the Hazard data
> The Hazard Comparison Dashboard is a prototype tool and a compilation of information sourced from many sites, databases and sources including U.S. Federal and state sources and international bodies that saves the user time by providing information in one location. The data are not reviewed by USEPA – the user must apply judgment in use of the information. The results do not indicate EPA’s position on the use or regulation of these chemicals. 